# ElasticSearch Obstoječi vektorji
Če imaš težave z računanjem vektorjev; Pokličeš Elasticsearch.   
Spet kličem vektorizacijo na stolpcu 'Text' in dobim stolpec z vektorjem 'Embedding'

In [45]:
!pip install pandas numpy networkx numpy scikit-learn

In [46]:
import os
import pandas as pd

data = pd.read_excel(os.path.join('..', 'data', 'Reworkan data 4.xlsx'))
data.head()

,Article ID,Datum,Dan,Teden,Article created,Tip medija,Podtip medija,Media uuid,Medij,Rubric uuid,...,Pregled objav - citiranost virov,Razredi Indeksa medijske podobe,Doseg,Naklonjenost teme,Osrednja tema,Ključna sporočila,Fotografija,Omemba v naslovu,Omemba v podnaslovu,Tekst
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Thursday,14,2023-04-06 01:31:33.498,tisk,tisk,2fd717ed-78ba-4f63-b257-cd096acb6bda,Delo,bc228842-98cc-48ea-8be9-0107becb707a,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"V spomin Prof. dr. Marija Pečan, dr. med. (193..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Friday,14,2023-04-07 13:47:41.072,internet,splet,754da261-9aee-4a1a-b9d8-734cd409fabf,Zurnal24.si,e059ba7d-1a91-48f6-ac1c-d7ed703375ba,...,Objava z obema tipoma citatov,"2. razred (34,5 - 59 točk)",odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"Člani Strateškega sveta za prehrano, ki je med..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Saturday,15,2023-04-15 21:44:58.961,internet,splet,bc20546f-3a11-4061-90c2-2769468cd542,Delo.si,f57a09ac-450d-4df4-bc2b-a2c7f96a4322,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Člani komisije se niso opredeljevali do znakov...
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Tuesday,14,2023-04-04 08:22:00.122,tisk,tisk,a67b08ee-a757-40ff-8d7e-ab8fc8246eec,Jana,4dd54f38-07e1-448f-b96d-dad85e669834,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),zelo dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Monday,16,2023-04-17 19:40:59.639,internet,splet,754da261-9aee-4a1a-b9d8-734cd409fabf,Zurnal24.si,e059ba7d-1a91-48f6-ac1c-d7ed703375ba,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"""Tudi vsi v zdravstvenem sistemu bodo dobili s..."


V realnem projektu, pazi na okoljske spremeljivke

In [47]:
print(f'CPTM_SPASS is set: {len(os.getenv("CPTM_SPASS"))}')
print(f'CPTM_SURL is set: {len(os.getenv("CPTM_SURL"))}')
print(f'CPTM_SUSER is set: {len(os.getenv("CPTM_SUSER"))}')

CPTM_SPASS is set: 20
CPTM_SURL is set: 61
CPTM_SUSER is set: 5


Funkcije za pridobivanje že obstoječih vektorjev iz elastike.

In [48]:
import numpy
import json
import requests
from requests.auth import HTTPBasicAuth

def embed_elastic(df: pd.DataFrame, col_name: str = 'Tekst', target_col_name: str = 'Embedding') -> pd.DataFrame:
    # Split the DataFrame into batches of 500 rows
    batch_size = 1000
    num_batches = (len(df) + batch_size - 1) // batch_size
    es_column = 'vector_768___textonic_v2'
    id_column = 'Article ID'
    
    responses = pd.DataFrame()
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        batch = df[id_column].iloc[start_idx:end_idx].tolist()
        
        query = '''
        {
          "query": {
            "bool": {
              "filter": [
                {"range": {"created": {"gte": "1996-01-01", "lt": "2030-01-01"}}},
                {"terms": {"uuid": ''' + json.dumps(batch) + '''}}
              ]
            }
          },
          "_source": ["uuid", ''' + json.dumps(es_column) + '''],
          "from": 0, "size": 1000, "sort": { "created": {"order": "asc"}}
        }
        '''
        
        result = {
            id_column: [],
            target_col_name: []
        }
        # Call the remote service for the current batch
        resp_text = ''
        try:
            resp = requests.post(os.getenv("CPTM_SURL"),
                                 headers={'Content-Type': 'application/json'},
                                 auth=HTTPBasicAuth(os.getenv("CPTM_SUSER"), os.getenv("CPTM_SPASS")),
                                 data=query)
            resp_text = json.loads(resp.text)
            for hit in resp_text['hits']['hits']:
                if es_column not in hit['_source']:
                    continue
                if 'uuid' not in hit['_source']:
                    continue
                vect = numpy.array(hit['_source'][es_column])  # here is article vector
                result[id_column].append(hit['_source']['uuid'])  # here is uuid of an article 
                result[target_col_name].append(vect)
            print(f'Requested/received/valid vector : {len(batch)}/{len(resp_text["hits"]["hits"])}/{len(result[target_col_name])}')
        except Exception as error:
            print(f'Elasticsearch request error [{error}] for query [{query}] with response [{resp_text}]')
            return df
        
        # Convert the collected responses to a DataFrame
        response_df = pd.DataFrame(result)
        responses = pd.concat([responses, response_df])
        
    # Merge the responses back into the original DataFrame
    orig_len = len(df)
    df = pd.merge(df, responses, on=id_column, how='left').dropna(subset=[target_col_name])
    print(f'Requested/Successful {orig_len}/{len(df)}')
    return df

Kličem vektorizacijo na stolpcu 'Text' in dobim stolpec z vektorjem 'Embedding'

In [49]:
data = embed_elastic(data, 'Tekst', 'Embedding')
data[['Article ID', 'Datum', 'Medij', 'Tekst', 'Embedding']].head(100)

Requested/received/valid vector : 1000/912/912
Requested/received/valid vector : 1000/924/924
Requested/received/valid vector : 1000/912/911
Requested/received/valid vector : 1000/888/887
Requested/received/valid vector : 991/920/920
Requested/Successful 4991/4989


,Article ID,Datum,Medij,Tekst,Embedding
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Delo,"V spomin Prof. dr. Marija Pečan, dr. med. (193...","[0.027611615136265755, 0.050394102931022644, 0..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Zurnal24.si,"Člani Strateškega sveta za prehrano, ki je med...","[0.028132325038313866, 0.07115303725004196, -0..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Delo.si,Člani komisije se niso opredeljevali do znakov...,"[0.024377623572945595, 0.04831267520785332, 0...."
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Jana,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit...","[0.006464230827987194, 0.04741932079195976, -0..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Zurnal24.si,"""Tudi vsi v zdravstvenem sistemu bodo dobili s...","[0.03661549463868141, 0.045082710683345795, 0...."
...,...,...,...,...,...
95,34ce443c-d2da-11ed-9f09-af460eec62b1,2023-04-04 13:16:00,Sta.si,Prodaja prehranskih dopolnil je vse bolj razši...,"[0.04838475584983826, 0.07178891450166702, -0...."
96,35d75476-e0da-11ed-9f09-af460eec62b1,2023-04-22 08:50:00,Rtvslo.si,Raziskujemo spanje. Vsi že poznamo spalne cikl...,"[0.03480922058224678, 0.04446624591946602, -0...."
97,35e5b4e9-d511-11ed-9f09-af460eec62b1,2023-04-07 08:53:00,Siol.net,"Termometer prikaže, kako vroč je članek. Skupn...","[0.010939517989754677, 0.03906986489892006, 0...."
98,3632728f-d881-11ed-bfff-ed625638e8e4,2023-04-11 15:31:00,24ur.com,SLOVENIJA\n'Parkinsonova bolezen v našem siste...,"[-0.0009582549682818353, 0.04825139418244362, ..."


Definiram funkcijo za cluster

In [50]:
import networkx as nx
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def cluster_louvain(df: pd.DataFrame, col_name: str = 'Embedding', similarity_threshold: float = 0.96):
    print("Clustering ...")
    embeddings = np.array(df[col_name].to_list())
    labels = [0] * len(embeddings)
    x = cosine_similarity(embeddings, embeddings)
    similarity_matrix = x > similarity_threshold
    graph = nx.from_numpy_array(similarity_matrix)
    communities = nx.algorithms.community.louvain_communities(graph, resolution=0.1)
    for community in communities:
        initial_member = min(community)
        for member in community:
            labels[member] = initial_member

    df['Cluster'] = pd.Series(labels)
    print("Clustered")
    return df

Izračunam cluster

In [51]:
data = cluster_louvain(data, 'Embedding', 0.92)  # <- similarity_threshold
data = data.drop(columns=['Embedding'])  # remove the vectors from data we don't need them anymore
data[['Article ID', 'Datum', 'Medij', 'Cluster', 'Tekst']].head(100)

Clustering ...
Clustered


,Article ID,Datum,Medij,Cluster,Tekst
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Delo,0.0,"V spomin Prof. dr. Marija Pečan, dr. med. (193..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Zurnal24.si,1.0,"Člani Strateškega sveta za prehrano, ki je med..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Delo.si,2.0,Člani komisije se niso opredeljevali do znakov...
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Jana,3.0,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Zurnal24.si,2.0,"""Tudi vsi v zdravstvenem sistemu bodo dobili s..."
...,...,...,...,...,...
95,34ce443c-d2da-11ed-9f09-af460eec62b1,2023-04-04 13:16:00,Sta.si,10.0,Prodaja prehranskih dopolnil je vse bolj razši...
96,35d75476-e0da-11ed-9f09-af460eec62b1,2023-04-22 08:50:00,Rtvslo.si,96.0,Raziskujemo spanje. Vsi že poznamo spalne cikl...
97,35e5b4e9-d511-11ed-9f09-af460eec62b1,2023-04-07 08:53:00,Siol.net,15.0,"Termometer prikaže, kako vroč je članek. Skupn..."
98,3632728f-d881-11ed-bfff-ed625638e8e4,2023-04-11 15:31:00,24ur.com,50.0,SLOVENIJA\n'Parkinsonova bolezen v našem siste...
